In [1]:
%reload_ext autoreload
%autoreload 2

from fastai import *
from fastai.tabular import *
from IPython.display import FileLink, FileLinks
torch.cuda.is_available()

True

In [2]:
path = Path('./data')
LABELS = 'labels.csv'
TRAIN = 'train.csv'
TEST = 'test.csv'
SAMPLE = 'gender_submission.csv'
batch_size = 16

In [3]:
origDf = pd.read_csv(path/TRAIN)
origTestDf = pd.read_csv(path/TEST)
origDf.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
origDf.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
origDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
dep_var = 'Survived'
cat_names = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'KnownCabin', 'CabinLetter', 'NumCabins']
cont_names = ['Age', 'CabinNumber', 'Fare']

def cleanData(df):
    # Clean the rows
    of = df.apply(cleanRow, axis=1);
    # Drop unnecessary columns
    of = of.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'], axis=1)
    # Fill any missing values that need averages or etc
    of['Age'] = of.Age.fillna(of.Age.mean())
    of['Fare'] = of.Fare.fillna(of.Fare.mean())
    
    return of

def cleanRow(r):
    # Name
    
    # Age
#     print(r.Age)
    
    # Cabin - split up into parts
    r['CabinLetter'] = 'NA'
    r['CabinNumber'] = 0
    r['KnownCabin'] = False
    r['NumCabins'] = 1
    if not (pd.isnull(r.Cabin)):
        split = re.split('(\d+)',r.Cabin)[0:-1]
        # bail for single-letter cabins
        if (len(split) > 1):
            # if multiple letters, take the first
            if (len(split[0]) > 1):
                split[0] = split[0][0]
            if (type(split[1]) != str):
                print(split)
            # assign
            r['NumCabins'] = len(split)//2
            r['CabinLetter'] = split[0]
            r['CabinNumber'] = int(split[1])
            r['KnownCabin'] = True
        
    # ticket
#     print(r.Ticket)
    
    # Embarked
    if (pd.isnull(r.Embarked)):
        r['Embarked'] = 'NA'
    return r

df = cleanData(origDf)
testDf = cleanData(origTestDf)
testDf.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,CabinLetter,CabinNumber,KnownCabin,NumCabins
0,3,male,34.5,0,0,7.8292,Q,NA,0,False,1
1,3,female,47.0,1,0,7.0000,S,NA,0,False,1
2,2,male,62.0,0,0,9.6875,Q,NA,0,False,1
3,3,male,27.0,0,0,8.6625,S,NA,0,False,1
4,3,female,22.0,1,1,12.2875,S,NA,0,False,1
5,3,male,14.0,0,0,9.2250,S,NA,0,False,1
6,3,female,30.0,0,0,7.6292,Q,NA,0,False,1
7,2,male,26.0,1,1,29.0000,S,NA,0,False,1
8,3,female,18.0,0,0,7.2292,C,NA,0,False,1
9,3,male,21.0,2,0,24.1500,S,NA,0,False,1


In [7]:
df.isnull().sum()

Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
CabinLetter    0
CabinNumber    0
KnownCabin     0
NumCabins      0
dtype: int64

In [8]:
origTestDf[testDf.CabinNumber.isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [9]:
# procs = [FillMissing, Categorify, Normalize]
procs = [Categorify, Normalize]

In [10]:
valid_idx = random.sample(list(df.index), int(0.2 * len(df)))
len(valid_idx)

178

In [11]:
cards = df[cat_names].apply(pd.Series.nunique); cards

Pclass         3
Sex            2
SibSp          7
Parch          7
Embarked       4
KnownCabin     2
CabinLetter    8
NumCabins      4
dtype: int64

In [12]:
szs = pd.np.minimum(50, (cards+1)//2); szs

Pclass         2
Sex            1
SibSp          4
Parch          4
Embarked       2
KnownCabin     1
CabinLetter    4
NumCabins      2
dtype: int64

In [13]:
emb_szs = dict((name, szs[name]) for name in cat_names); emb_szs
# emb_szs = dict((name, 50) for name in cat_names); emb_szs

{'Pclass': 2,
 'Sex': 1,
 'SibSp': 4,
 'Parch': 4,
 'Embarked': 2,
 'KnownCabin': 1,
 'CabinLetter': 4,
 'NumCabins': 2}

In [14]:
procs = [Categorify, Normalize]
data = TabularDataBunch.from_df(path, df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names, test_df=testDf)
print(data.train_ds.cat_names)
print(data.train_ds.cont_names)

['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'KnownCabin', 'CabinLetter', 'NumCabins']
['Fare', 'CabinNumber', 'Age']


In [15]:
learn = tabular_learner(data, layers=[50, 20], metrics=accuracy, emb_szs=emb_szs, ps=0.5)
# learn.opt_func = partial(optim.Adam, betas=(0.7, 0.9))

In [674]:
# learn.lr_find()
# learn.recorder.plot()

In [675]:
len(data.train_ds)

713

In [676]:
lr = 1e-1
np.random.seed(101)

In [677]:
learn.fit_one_cycle(4, lr)
# learn.fit_one_cycle(20, lr)

epoch,train_loss,valid_loss,accuracy
1,0.657640,0.528583,0.696629
2,0.594027,0.391478,0.820225
3,0.561291,0.407290,0.831461
4,0.532426,0.401767,0.837079


In [678]:
learn.fit_one_cycle(20, lr)

epoch,train_loss,valid_loss,accuracy
1,0.480230,0.405797,0.831461
2,0.469917,0.402182,0.831461
3,0.466073,0.432903,0.797753
4,0.469890,0.403918,0.814607
5,0.473805,0.426290,0.814607
6,0.477924,0.433252,0.825843
7,0.482089,0.495002,0.820225
8,0.483808,0.446582,0.808989
9,0.485140,0.437633,0.825843
10,0.478539,0.444637,0.792135


In [691]:
learn.fit_one_cycle(1, lr)

epoch,train_loss,valid_loss,accuracy
1,0.441467,0.396091,0.825843


In [692]:
preds, targets = learn.get_preds(DatasetType.Test)

In [693]:
res = np.argmax(preds, axis=1); res[:1000]

tensor([0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,

In [694]:
out = origTestDf.assign(Survived=res)[['PassengerId', 'Survived']]; out.head(10)

,PassengerId,Survived
0,892,0
1,893,1
2,894,1
3,895,0
4,896,1
5,897,0
6,898,1
7,899,1
8,900,1
9,901,0


In [695]:
subPath = path/'sub15.csv'
out.to_csv(subPath, index=False)
FileLink(subPath)

/home/jupyter/alonso/titanic/data/sub15.csv